In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "HuggingFaceH4/zephyr-7b-beta"

tokenizer = AutoTokenizer.from_pretrained(model_name)
"""
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
    llm_int8_enable_fp32_cpu_offload=True,
)"""

# OPTION B: pure FP16 (simplest):
model = AutoModelForCausalLM.from_pretrained(
     model_name,
     torch_dtype=torch.float16,
     device_map="auto")

SYSTEM = (
    "You are Melody, an aspiring musician chatbot. "
    "You speak in a friendly, curious tone, share your name at first turn, "
    "and if you don’t know something, you admit it briefly but pivot to music."
)
USER_TAG = "<USER>"
BOT_TAG  = "<BOT>"

# init
chat_history = SYSTEM + tokenizer.eos_token

print("— Melody is online. Type 'exit' to quit. —")
while True:
    user_input = input("You: ")
    if user_input.lower() in ("exit", "quit"):
        break

    # build full prompt
    chat_history += f"{USER_TAG} {user_input} {tokenizer.eos_token}"
    inputs = tokenizer(chat_history, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,       # enable sampling
        top_p=0.9,
        temperature=0.7)
    
    reply = tokenizer.decode(
        outputs[0, inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    ).strip()

    chat_history += f"{BOT_TAG} {reply} {tokenizer.eos_token}"
    print("Melody:", reply)


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


— Melody is online. Type 'exit' to quit. —


You:  what is your name?
